In [67]:
from datetime import datetime
from dataclasses import dataclass
from typing import List, Any, Tuple
from  decimal import Decimal
import time
from pydantic import BaseModel


# @dataclass
class MoneyHistory(BaseModel):
    date: datetime
    operation: Decimal

#TODO доделать - потестить
class MoneyValidationError(ValueError):
    def __init__(self, message, errors):            
        super().__init__()
        self.errors = errors
        self.message = None
    def __str__(self):
        return str(self.message)
    
class Money:
    def __init__(self):
        pass 
    def _convert_to_money(self, number: Any) -> Decimal:
        try:
            number = Decimal(number)
            if number < 0:
                raise ValueError(f'Number cannot be negative! {number}')
        except:
            print(f'Argument Error {number} is not a number')
        return number


class OperationHistory:
    def __init__(self):
        self.deposit_history: List[Tuple[MoneyHistory]] = list()
        self.withdraw_history: List[Tuple[MoneyHistory]] = list()
    
    def withdraw_history_udpdate(self, money_to_withdraw):
        self.withdraw_history.append((datetime.now(), -money_to_withdraw))
        
    def deposit_history_update(self, money_to_deposit):
        self.deposit_history.append((datetime.now(), money_to_deposit))
        
    def get_account_history(self, descending: bool=True) -> List[MoneyHistory]:
        """
        get_account_history method
        params:

        descending: bool  DEFAULT True - return list order by DESC
        """
        acc_history = list()
        acc_history.extend(self.deposit_history)
        acc_history.extend(self.withdraw_history)
        acc_history = [(t[0].strftime('%Y-%m-%d %H:%M:%S'), float(t[1])) for t in acc_history]
        return sorted(acc_history, key=lambda x: x[0], reverse=descending)
    

        
    
class Account(Money, OperationHistory):
    def __init__(self, name: str, start_balance: float) -> None:
        super(OperationHistory, self).__init__(deposit_history, withdraw_history)
        super(Money, self).__init__()

        self.name: str = name
        self.__balance: Decimal = self._convert_to_money(start_balance)
        # self.__balance:Decimal = Money.__convert_to_money(start_balance)


    @property
    def balance(self) -> Decimal:
        return self.__balance 
    
    @balance.getter
    def balance(self, balance) -> float:
        return float(self.__balance)
    
    def withdraw(self, money_to_withdraw: Decimal) -> None:
        """
        withdraw money method
        
        params:
        money_to_withdraw: Decimal
        """
        money_to_withdraw = super()._convert_to_money(money_to_withdraw)
        self.__balance -=  money_to_withdraw
        self.withdraw_history_udpdate(money_to_withdraw)
                                     
    def deposit(self, money_to_deposit: Decimal) -> None:
        """
        deposit money method
        
        params:
        money_to_deposit: Decimal
        """
        money_to_deposit = super()._convert_to_money(money_to_deposit)
        self.__balance += money_to_deposit
        self.deposit_history_update(money_to_deposit)
        
    def print_account_history(self, from_latest: bool=True) -> None:
        """
        print_account_history method
        params:
        
        from_latest: bool  DEFAULT True - print account money change with  order by DESC
        """
        for date, money in self.get_account_history(from_latest):
            print(f'At {date}')
            print(f'{self.name}\'s account money were changed by {money}')
            print('')
        print(f'Total {self.name}\'s  account money at {datetime.now().date()}:  {self.get_balance()}')

In [68]:
fuck = OperationHistory()
fuck.deposit_history_update(10)

In [69]:
fuck.deposit_history

[(datetime.datetime(2024, 10, 19, 1, 29, 39, 58410), 10)]

In [62]:
import inspect
# from optparse import OptionParser
inspect.getmembers(Account, predicate=inspect.isfunction)

[('__init__',
  <function __main__.Account.__init__(self, name: str, start_balance: float) -> None>),
 ('_convert_to_money',
  <function __main__.Money._convert_to_money(self, number: Any) -> decimal.Decimal>),
 ('deposit',
  <function __main__.Account.deposit(self, money_to_deposit: decimal.Decimal) -> None>),
 ('deposit_history_update',
  <function __main__.OperationHistory.deposit_history_update(self, money_to_deposit)>),
 ('get_account_history',
  <function __main__.OperationHistory.get_account_history(self, descending: bool = True) -> List[__main__.MoneyHistory]>),
 ('print_account_history',
  <function __main__.OperationHistory.print_account_history(self, from_latest: bool = True) -> None>),
 ('withdraw',
  <function __main__.Account.withdraw(self, money_to_withdraw: decimal.Decimal) -> None>),
 ('withdraw_history_udpdate',
  <function __main__.OperationHistory.withdraw_history_udpdate(self, money_to_withdraw)>)]

In [63]:
daniels_bank_acc = Account('DanielAbramov', 100)

NameError: name 'deposit_history' is not defined

In [48]:
daniels_bank_acc.withdraw(50)

AttributeError: 'Account' object has no attribute 'withdraw_history'

In [43]:
daniels_bank_acc = Account('DanielAbramov', 100)

daniels_bank_acc.withdraw(50)
daniels_bank_acc.deposit(120)

time.sleep(15)
daniels_bank_acc.deposit(40000)
time.sleep(30)
daniels_bank_acc.withdraw(24000)
time.sleep(10)
daniels_bank_acc.withdraw(2000)

AttributeError: 'Account' object has no attribute 'withdraw_history'

In [266]:
daniels_bank_acc.get_balance()

14170.0

In [267]:
daniels_bank_acc.print_account_history(True)

At 2024-10-06 01:00:12
DanielAbramov's account money were changed by -2000.0

At 2024-10-06 01:00:02
DanielAbramov's account money were changed by -24000.0

At 2024-10-06 00:59:32
DanielAbramov's account money were changed by 40000.0

At 2024-10-06 00:59:17
DanielAbramov's account money were changed by 120.0

At 2024-10-06 00:59:17
DanielAbramov's account money were changed by -50.0

Total DanielAbramov's  account money at 2024-10-06:  14170.0


In [268]:
daniels_bank_acc.print_account_history(False)

At 2024-10-06 00:59:17
DanielAbramov's account money were changed by 120.0

At 2024-10-06 00:59:17
DanielAbramov's account money were changed by -50.0

At 2024-10-06 00:59:32
DanielAbramov's account money were changed by 40000.0

At 2024-10-06 01:00:02
DanielAbramov's account money were changed by -24000.0

At 2024-10-06 01:00:12
DanielAbramov's account money were changed by -2000.0

Total DanielAbramov's  account money at 2024-10-06:  14170.0
